# **Twitter Step 3: Exploratory Data Analysis (EDA) and Preliminary Visualizations**

* Explore the collected and cleaned dataset
* Do some basic visualizations to understand the dataset characteristics

Reference:

* https://www.freecodecamp.org/news/how-to-combine-multiple-csv-files-with-8-lines-of-code-265183e0854/

# **0. Setup Environment**

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

     |████████████████████████████████| 266kB 5.5MB/s 
     |████████████████████████████████| 604kB 16.3MB/s 
     |████████████████████████████████| 71kB 6.5MB/s 
     |████████████████████████████████| 3.1MB 20.5MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 112kB 38.7MB/s 
     |████████████████████████████████| 296kB 43.3MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27085 sha256=54bfcfaf998f31cc1b413334f877d995b40441a1b5a513640a7b7a3cee0582bc
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
Successfully built htmlmin
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5
  Found existing installation: pandas-profiling 1.4.1
    Uninstal

## You want to restart the kernel at this point (watch for warning in output of last code cell above)

In [1]:
from pathlib import Path

import requests

import pandas_profiling
from pandas_profiling.utils.cache import cache_file

## You will need to give permission for this Colab to link to your gdrive in the code cell below

In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [3]:
# CUSTOMIZE: if you want your work and twitter datasets saved into a specific folder
#            beneath your gdrive root directory, define it below

%cd ./MyDrive/courses/2020f_iphs200_programming_humanity/code/twint/

/gdrive/MyDrive/courses/2020f_iphs200_programming_humanity/code/twint


In [4]:
!ls *.csv

cleaned_tweets_combined_20201204-193123.csv
tweets_combined_20201204-193111.csv
tweets_combined_20201204-193123.csv
tweets_twint_tesla__20201204-192926.csv


In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [7]:
import os
import re
import glob


# **1. Read in Combined Tweet Dataset File**

In [8]:
!ls *.csv

cleaned_tweets_combined_20201204-193123.csv
tweets_combined_20201204-193111.csv
tweets_combined_20201204-193123.csv
tweets_twint_tesla__20201204-192926.csv


In [9]:
# CONFIGURE: Set the 'file_name_all' to the name of the combined datafile with all the tweets
#            which should be listed in the previous code cell

file_name_all = 'cleaned_tweets_combined_20201204-193123.csv'

In [10]:
combined_df = pd.read_csv(file_name_all, encoding='utf-8')
combined_df = combined_df.convert_dtypes()
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1591 entries, 0 to 1590
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       1591 non-null   Int64  
 1   id               1591 non-null   Int64  
 2   conversation_id  1591 non-null   Int64  
 3   created_at       1591 non-null   string 
 4   date             1591 non-null   string 
 5   time             1591 non-null   string 
 6   timezone         1591 non-null   Int64  
 7   user_id          1591 non-null   Int64  
 8   username         1591 non-null   string 
 9   name             1591 non-null   string 
 10  place            0 non-null      Int64  
 11  tweet            1591 non-null   string 
 12  language         1591 non-null   string 
 13  mentions         1591 non-null   string 
 14  urls             1591 non-null   string 
 15  photos           1591 non-null   string 
 16  replies_count    1591 non-null   Int64  
 17  retweets_count

In [11]:
combined_df.shape

(1591, 41)

In [13]:
combined_df = combined_df.convert_dtypes()
combined_df.head()

,Unnamed: 0,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,emojis,smileys,numbers,tweet_clean
0,0,1334127507906314240,1333882505775050752,2020-12-02 13:29:19+00:00,2020-12-02,13:29:19,0,44196397,elonmusk,Elon Musk,<NA>,"@Tesmanian_com Award accepted on behalf of the great people at Tesla, SpaceX, Neuralink &amp; Boring Co",en,['@Tesmanian_com'],[],[],313,284,8385,[],[],https://twitter.com/elonmusk/status/1334127507906314240,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"[{'screen_name': 'Tesmanian_com', 'name': 'Tesmanian.com', 'id': '1100520274200416256'}]",<NA>,<NA>,<NA>,<NA>,[],[],[],award accepted on behalf of the great people at tesla spacex neuralink &amp boring co
1,1,1331075177262661633,1330843965613027329,2020-11-24 03:20:27+00:00,2020-11-24,03:20:27,0,44196397,elonmusk,Elon Musk,<NA>,"@PPathole @Teslarati @TeslaRoadTrip We’re still far from simply video in, control out. The biggest game-changer, currently underway at Tesla, is 360 degree, high fps video for labeling, training &amp; inference.",en,"['@PPathole', '@Teslarati', '@TeslaRoadTrip']",[],[],135,97,2542,[],[],https://twitter.com/elonmusk/status/1331075177262661633,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"[{'screen_name': 'PPathole', 'name': 'Pranay Pathole', 'id': '1291945442'}, {'screen_name': 'Teslarati', 'name': 'TESLARATI', 'id': '1308211178'}, {'screen_name': 'TeslaRoadTrip', 'name': 'TeslaRoadTrip', 'id': '1182382878'}]",<NA>,<NA>,<NA>,<NA>,[],[],[' 360'],we are still far from simply video in control out the biggest game changer currently underway at tesla is degree high fps video for labeling training &amp inference
2,2,1330982572038500355,1330980211509186560,2020-11-23 21:12:28+00:00,2020-11-23,21:12:28,0,44196397,elonmusk,Elon Musk,<NA>,@vincent13031925 @Tesla Wow,und,"['@vincent13031925', '@Tesla']",[],[],790,367,19411,[],[],https://twitter.com/elonmusk/status/1330982572038500355,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"[{'screen_name': 'vincent13031925', 'name': 'Vincent 🚀\U0001f7e0', 'id': '1689516060'}, {'screen_name': 'Tesla', 'name': 'Tesla', 'id': '13298072'}]",<NA>,<NA>,<NA>,<NA>,[],[],[],wow
3,3,1330206139385044999,1330179916587884544,2020-11-21 17:47:12+00:00,2020-11-21,17:47:12,0,44196397,elonmusk,Elon Musk,<NA>,@heydave7 @philwhln Tesla is a vehicle for creating &amp; producing many useful products,en,"['@heydave7', '@philwhln']",[],[],103,106,1941,[],[],https://twitter.com/elonmusk/status/1330206139385044999,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"[{'screen_name': 'heydave7', 'name': 'Dave Lee', 'id': '29893444'}, {'screen_name': 'philwhln', 'name': 'Phil Whelan 😷🇨🇦\U0001f995🚀', 'id': '13036132'}]",<NA>,<NA>,<NA>,<NA>,[],[],[],tesla is a vehicle for creating &amp producing many useful products
4,4,1330187635483140099,1330179916587884544,2020-11-21 16:33:40+00:00,2020-11-21,16:33:40,0,44196397,elonmusk,Elon Musk,<NA>,"@philwhln @heydave7 Because I am not an investor. Tesla is definitely not the only good company, but investing is not what I do. But I always put my own money into companies I help create, otherwise it’d be wrong to ask others to do so.",en,"['@philwhln', '@heydave7']",[],[],130,145,2495,[],[],https://twitter.com/elonmusk/status/1330187635483140099,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"[{'screen_name': 'philwhln', 'name': 'Phil Whelan 😷🇨🇦\U0001f995🚀', 'id': '13036132'}, {'screen_name': 'heydave7', 'name': 'Dave Lee', 'id': '29893444'}]",<NA>,<NA>,<NA>,<NA>,[],[],[],because i am not an investor tesla is definitely not the only good company but investing is not what i do but i always put my own money into companies i help create otherwise it would be wrong to ask others to do so


# **2. Pandas-Profiling Generated Report**
By: Jon Chun
30 Nov 2020

Lo/No-code solution for Pandas EDA and simple visualizations.
References:

* https://github.com/pandas-profiling/pandas-profiling
* https://colab.research.google.com/github/pandas-profiling/pandas-profiling/blob/master/examples/meteorites/meteorites.ipynb#scrollTo=KbIMcOpp_rfh


In [14]:
# Option #1 to display Pandas-Profiling Report

report = combined_df.profile_report(
    sort="None", html={"style": {"full_width": True}}, progress_bar=False
)
report

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
combined_df.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,emojis,smileys,numbers,tweet_clean
0,1332356759994970113,1332356759994970113,2020-11-27 16:13:00+00:00,2020-11-27,16:13:00,0,939091,joebiden,Joe Biden,<NA>,"This Native American Heritage Day, we give thanks to our Indigenous communities and their ancestors. As we celebrate their rich heritage and contributions, let’s commit to writing a new future together — one built on a strong partnership and filled with opportunity for all.",en,[],[],[],6351,21601,248367,[],[],https://twitter.com/JoeBiden/status/1332356759994970113,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>,[],[],[],this native american heritage day we give thanks to our indigenous communities and their ancestors as we celebrate their rich heritage and contributions let us commit to writing a new future together one built on a strong partnership and filled with opportunity for all
1,1323473727447654400,1323473727447654400,2020-11-03 03:55:00+00:00,2020-11-03,03:55:00,0,939091,joebiden,Joe Biden,<NA>,I’ve said it many times: I’m more optimistic about America’s future today than I was when I got elected to the United States Senate as a 29-year-old.,en,[],[],[],4359,5375,79312,[],[],https://twitter.com/JoeBiden/status/1323473727447654400,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>,[],[],[ 29],i have said it many times i am more optimistic about americas future today than i was when i got elected to the united states senate as a year old
2,1323391493885718528,1323391493885718528,2020-11-02 22:28:14+00:00,2020-11-02,22:28:14,0,939091,joebiden,Joe Biden,<NA>,I’m speaking with members of the African American community in Pittsburgh about the power of the vote — and the future we can build together. Tune in. https://t.co/1wFBiLoCWu,en,[],[https://t.co/1wFBiLoCWu],[],1818,1994,13823,[],[],https://twitter.com/JoeBiden/status/1323391493885718528,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>,[],[],[],i am speaking with members of the african american community in pittsburgh about the power of the vote and the future we can build together tune in
3,1322959086544211973,1322959086544211973,2020-11-01 17:50:00+00:00,2020-11-01,17:50:00,0,939091,joebiden,Joe Biden,<NA>,We can build a future where: - Health care is a right - We end the gun violence epidemic - We combat climate change - Our government works for everyone Vote. https://t.co/eoxT07d7QB,en,[],[https://t.co/eoxT07d7QB],[],1603,2581,18087,[],[],https://twitter.com/JoeBiden/status/1322959086544211973,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>,[],[],[],we can build a future where health care is a right we end the gun violence epidemic we combat climate change our government works for everyone vote
4,1322927509260902401,1322927509260902401,2020-11-01 15:44:31+00:00,2020-11-01,15:44:31,0,939091,joebiden,Joe Biden,<NA>,The future of our planet is on the ballot. Vote: https://t.co/eoxT07d7QB,en,[],[https://t.co/eoxT07d7QB],[],2155,4513,43222,[],[],https://twitter.com/JoeBiden/status/1322927509260902401,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>,[],[],[],the future of our planet is on the ballot vote


# **3. NLP Analysis**

Options

* Sentiment Analysis (FastText and/or VADER)
* Named Entity Recognition (NER using flair)

# **3. Write Reports to Output files**

In [15]:
# Create unique output filename using current datetime stamp

file_name_edaviz = ('edaviz_' + file_name_all.split('.')[0] + '.html') # 'tweets_combined_20201201-012404.csv'
print(file_name_edaviz)

edaviz_cleaned_tweets_combined_20201204-193123.html


In [16]:
profile_report = combined_df.profile_report(html={"style": {"full_width": True}})
profile_report.to_file(file_name_edaviz)

In [17]:
!ls eda_viz*

ls: cannot access 'eda_viz*': No such file or directory


In [18]:
# Option #2 to display Pandas-Profiling Report

# import IPython
# IPython.display.HTML(filename=file_name_edaviz)

In [19]:
!pwd

/gdrive/My Drive/courses/2020f_iphs200_programming_humanity/code/twint


In [20]:
!ls *.csv

cleaned_tweets_combined_20201204-193123.csv
tweets_combined_20201204-193111.csv
tweets_combined_20201204-193123.csv
tweets_twint_tesla__20201204-192926.csv


In [ ]:
# Option #3 to display Pandas-Profiling Report

"""
profile_report = combined_df.profile_report(
    explorative=True, html={"style": {"full_width": True}}
)
profile_report
"""